In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold,GroupKFold
import math
import scipy
import numpy as np
import pandas as pd
import sklearn.metrics as mtr 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout, PReLU, BatchNormalization, ELU, GaussianNoise, Activation
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
from sklearn import preprocessing
import gc
import os
from tqdm import tqdm_notebook
import keras.backend as K
#note： 
#1. As a result it might not be worthwhile to use features related to game clock/quarter of the game。
#2. There is no relationships between number of rushes before and running yards gained。
#3. rushing success larger depends on defender in box, or defender that are close to offensive lineman and attempt 
#   to counter the blocking.
#4. highly drafted player has the same average rushing yards as the rest

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_df = pd.read_csv(r'\\Neptune\RiskMgmtArchive\3 Temp Hold\Temp_Jason\research\train.csv')

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#team abbreviations correct
def data_clean(df):
#correct name   
    df.loc[df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
    df.loc[df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
    df.loc[df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
    df.loc[df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
    df.loc[df['FieldPosition'] == 'ARZ', 'FieldPosition'] = 'ARI'
    df.loc[df['FieldPosition'] == 'BLT', 'FieldPosition'] = 'BAL'
    df.loc[df['FieldPosition'] == 'CLV', 'FieldPosition'] = 'CLE'
    df.loc[df['FieldPosition'] == 'HST', 'FieldPosition'] = 'HOU'


# offense time and defence time
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    
#time
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df['GameClock'] = df['GameClock'].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
#player height
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

#weather
    def map_weather(txt):
        ans = 1
        if pd.isna(txt):
            return 0
        if 'partly' in txt:
            ans*=0.5
        if 'climate controlled' in txt or 'indoor' in txt:
            return ans*3
        if 'sunny' in txt or 'sun' in txt:
            return ans*2
        if 'clear' in txt:
            return ans
        if 'cloudy' in txt:
            return -ans
        if 'rain' in txt or 'rainy' in txt:
            return -2*ans
        if 'snow' in txt:
            return -3*ans
        return 0
    df['Cleaned_GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['Cleaned_GameWeather'].apply(map_weather)

#diff Score    
    df["DiffScoreBeforePlay_ob"] = (df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"])
    df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]] = - df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]]
#Turf
    def agrupar_gramado(Turf):
        if Turf == 'Artifical':
            return 'Artificial'

        elif Turf in ('FieldTurf', 'Field turf'):
            return 'Field Turf'

        elif Turf in ('FieldTurf360', 'FieldTurf 360'):
            return 'Field Turf 360'

        elif Turf in ('Natural', 'Natural grass', 'Naturall Grass', 'grass', 'natural grass', 'SISGrass', 'Natural Grass'):
            return "Grass"

        elif Turf == "UBU Sports Speed S5-M":
            return "UBU Speed Series-S5-M"

        else:
            return Turf
    df['Turf'] = df['Turf'].apply(agrupar_gramado)

#left to right
    df['New_X'] = df['X']
    df.loc[df['PlayDirection'] == 'left','New_X'] = 120 - df.loc[df['PlayDirection'] == 'left','X']
    df['New_Y'] = df['Y']
    df.loc[df['PlayDirection'] == 'left','New_Y'] = 160/3 - df.loc[df['PlayDirection'] == 'left','Y']
    df['Orientation_std'] = df['Orientation']
    df.loc[df['Season'] == 2017, 'Orientation_std'] = df.loc[df['Season'] == 2017, 'Orientation_std'] + 90
    #df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std'] = 360 - df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std']
    df.loc[df['PlayDirection'] == 'left', 'Orientation_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation_std'], 360)
    df['Dir_std'] = df['Dir']
    #df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = 360 - df.loc[df['PlayDirection'] == 'left', 'Dir_std']
    df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir_std'], 360)
    df['YardLine_std'] = 100 - df['YardLine']
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine_std'
         ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine']
    df["Orientation_sin"] = df["Orientation_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Orientation_cos"] = df["Orientation_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    df["Dir_sin"] = df["Dir_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Dir_cos"] = df["Dir_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

#distance and S
    
    #distance to yardline
    df['Dis_YardLine'] = df['New_X'] - df['YardLine_std'] - 10
    
    #distance to rusher
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    Rusher =df.loc[df['IsRusher'],['PlayId','X','Y','Dir_std','S']].rename(columns={"X":"Rusher_X","Y":"Rusher_Y",'PossessionTeam':'Offense_Team','Dir_std':'Rusher_Dir_std','S':'Rusher_Speed'})
    df = df.merge(Rusher,how = 'left',on = 'PlayId')
    df['Distance_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_Diff'] = df['Degree_to_Rusher'] - df['Rusher_Dir_std']
    df['Degree_Diff2'] = 270 - (90-df['Degree_to_Rusher']) - df['Dir_std']
    df['Speed_Ratio'] =  (df['Rusher_Speed']+0.01)/(df['S']+0.01)

#back_direction
#    def back_direction(orientation):
#        if orientation > 180.0:
#            return 1
#        else:
#            return 0
#    df['back_oriented_down_field'] = df['Orientation_std'].apply(lambda x: back_direction(x))

# Stadium and location clean
#    def agrupar_local(Location):
#        if Location == "Arlington, Texas":
#            return "Arlington, TX"
#        elif Location in ("Baltimore, Maryland","Baltimore, Md."):
#            return "Baltimore, MD"
#        elif Location == "Charlotte, North Carolina":
#            return "Charlotte, NC"
#        elif Location == "Chicago. IL":
#            return "Chicago, IL"
#        elif Location == "Cincinnati, Ohio":
#            return "Cincinnati, OH"
#        elif Location in ("Cleveland","Cleveland Ohio","Cleveland, Ohio","Cleveland,Ohio"):
#            return "Cleveland, OH"
#        elif Location == "Detroit":
#            return "Detroit, MI"
#        elif Location == "E. Rutherford, NJ" or Location == "East Rutherford, N.J.":
#            return "East Rutherford, NJ"
#        elif Location == "Foxborough, Ma":
#            return "Foxborough, MA"
#        elif Location == "Houston, Texas":
#            return "Houston, TX"
#        elif Location in ("Jacksonville Florida","Jacksonville, Fl","Jacksonville, Florida"):
#            return "Jacksonville, FL"
#        elif Location == "London":
#            return "London, England"
#        elif Location == "Los Angeles, Calif.":
#            return "Los Angeles, CA"
#        elif Location == "Miami Gardens, Fla.":
#            return "Miami Gardens, FLA"
#        elif Location in ("New Orleans","New Orleans, La."):
#            return "New Orleans, LA"
#        elif Location == "Orchard Park NY":
#            return "Orchard Park, NY"
#        elif Location == "Philadelphia, Pa.":
#            return "Philadelphia, PA"
#        elif Location == "Pittsburgh":
#            return "Pittsburgh, PA"
#        elif Location == "Seattle":
#            return "Seattle, WA"
#        else:
#            return Location

#    df['Location'] = df['Location'].apply(agrupar_local)

# stadium types
#    def agrupar_tipo_estadio(StadiumType):
#        outdoor       = ['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
#        indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
#        indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
#        dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
#        dome_open     = ['Domed, Open', 'Domed, open']

#        if StadiumType in outdoor:
#            return 'outdoor'
#        elif StadiumType in indoor_closed:
#            return 'indoor_closed'
#        elif StadiumType in indoor_open:
#            return 'indoor_open'
#        elif StadiumType in dome_closed:
#            return 'dome_closed'
#        elif StadiumType in dome_open:
#            return 'dome_open'
#        else:
#            return 'unknown'
#    df['StadiumType'] = df['StadiumType'].apply(agrupar_tipo_estadio)

# wind 
#    def give_me_WindSpeed(x):
#            x = str(x)
#            x = x.replace('mph', '').strip()
#            if '-' in x:
#                x = (int(x.split('-')[0]) + int(x.split('-')[1])) / 2
#            elif 'gusts up to' in x:
#                x = (int(x.split()[0]) + int(x.split()[-1])) / 2
#            elif 'clam' in x:
#                x = 0
#            try:
#                return float(x)
#            except:
#                return -99
#    df['Cleaned_WindSpeed'] = df['WindSpeed'].apply(give_me_WindSpeed)

# wind direction
#    def agrupa_wind_direction(WindDirection):
#        wd = str(WindDirection).upper()
#
#        if wd == 'N' or 'FROM N' in wd:
#            return 'north'
#        if wd == 'S' or 'FROM S' in wd:
#            return 'south'
#        if wd == 'W' or 'FROM W' in wd:
#            return 'west'
#        if wd == 'E' or 'FROM E' in wd:
#            return 'east'

#       if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
#            return 'south west'
#        if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
#            return 'south east'
#        if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
#            return 'north west'
#        if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
#            return 'north east'

#        if 'NW' in wd or 'NORTHWEST' in wd:
#            return 'north west'
#        if 'NE' in wd or 'NORTH EAST' in wd:
#            return 'north east'
#        if 'SW' in wd or 'SOUTHWEST' in wd:
#            return 'south west'
#        if 'SE' in wd or 'SOUTHEAST' in wd:
#            return 'south east'
#
#        return 'unknown'
    
#    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)

# speed
    df.loc[df['Season'] == 2017, 'S'] = (df.loc[df['Season'] == 2017, 'S'] - 2.4355) / 1.2930 * 1.4551 + 2.7570
    df['Horizontal Speed'] = df['S']*df["Dir_sin"]
    df['Vertical Speed'] = df['S']*df["Dir_cos"]
# momentum
#    df['momentum'] = df['S']*df['PlayerWeight']

    
    return df

In [4]:
train = data_clean(train_df)

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in remainder
C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in remainder
C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in double_scalars


In [97]:
train['Dir_std'].isnull().sum()

14

In [124]:
temp = train.drop(train[(train['Position'].isin(['FS','S','SS'])) & (train['Distance_to_Rusher']>15)].index)
temp = temp.merge(train.loc[train['IsRusher'],['PlayId','New_X','New_Y','Dir_std']]\
                  .rename(columns={'New_X':'Rusher_X','New_Y':'Rusher_Y','Dir_std':'Rusher_Dir'}), on = 'PlayId',how = 'left')

#temp2 = temp.loc[(temp['IsOnOffense'])]
temp = temp.loc[(~temp['IsOnOffense'])]


op_up = temp[((temp['Rusher_Dir']<90)|(temp['Rusher_Dir']>270))&(temp['New_Y']>=temp['Rusher_Y_x'])]
op_up = op_up.sort_values(['PlayId','New_Y'])
op_up[['Next_Y','Next_X']] = op_up[['New_Y','New_X']].shift(1,axis = 0)
op_up['Gap'] = op_up['New_Y'] - op_up['Next_Y']
op_up[['PlayId','New_Y','Next_Y','Gap']]
closest_op_up = op_up[['PlayId','Gap']].groupby('PlayId').head(1).reset_index()
best_up = op_up[['PlayId','Gap']].groupby('PlayId').max().reset_index()
def_num_up = op_up[['PlayId','Gap']].rename(columns={'Gap': 'num_def'}).groupby('PlayId').count().reset_index()

op_down = temp[(~((temp['Rusher_Dir']<90)|(temp['Rusher_Dir']>270)))&(temp['New_Y']<temp['Rusher_Y_x'])]
op_down = op_down.sort_values(['PlayId','New_Y'])
op_down[['Next_Y','Next_X']] = op_down[['New_Y','New_X']].shift(1,axis = 0)
op_down['Gap'] = op_down['New_Y'] - op_down['Next_Y']
op_down[['PlayId','New_Y','Next_Y','Gap']]
closest_op_down = op_down[['PlayId','Gap']].groupby('PlayId').tail(1).reset_index()
best_down = op_down[['PlayId','Gap']].groupby('PlayId').max().reset_index()
def_num_down = op_down[['PlayId','Gap']].rename(columns={'Gap': 'num_def'}).groupby('PlayId').count().reset_index()

#of_num_up = temp2[((temp2['Rusher_Dir']<90)|(temp2['Rusher_Dir']>270))&(temp2['New_Y']>=temp2['Rusher_Y_x'])]
#of_num_up = of_num_up[['PlayId','New_Y']].rename(columns={'New_Y': 'num_of'}).groupby('PlayId').count().reset_index()

#of_num_down = temp2[(~((temp2['Rusher_Dir']<90)|(temp2['Rusher_Dir']>270)))&(temp2['New_Y']<temp2['Rusher_Y_x'])]
#of_num_down = of_num_down[['PlayId','New_Y']].rename(columns={'New_Y': 'num_of'}).groupby('PlayId').count().reset_index()


closest_op = closest_op_up.append(closest_op_down)
best = best_up.append(best_down)
#def_num = def_num_up.append(def_num_down)
#of_num = of_num_up.append(of_num_down)
#def_of = def_num.merge(of_num, on = 'PlayId', how = 'left')
#def_of['def-of'] = def_of['num_def'] - def_of['num_of']
#print(def_of)

               PlayId  num_def  num_of  def-of
0      20170907000118        1     2.0    -1.0
1      20170907000189       10    11.0    -1.0
2      20170907000516        7     8.0    -1.0
3      20170907000917        7     8.0    -1.0
4      20170907001077        2     2.0     0.0
5      20170907001156        9    10.0    -1.0
6      20170907001488        5     6.0    -1.0
7      20170907001509        4     5.0    -1.0
8      20170907001530        4     6.0    -2.0
9      20170907001605        4     6.0    -2.0
10     20170907001715        1     NaN     NaN
11     20170907002648        4     4.0     0.0
12     20170907002829        3     3.0     0.0
13     20170907002900        5     8.0    -3.0
14     20170907003444        4     4.0     0.0
15     20170907003465        5     6.0    -1.0
16     20170907003507        4     4.0     0.0
17     20170907004046        6     9.0    -3.0
18     20170907004182        5     7.0    -2.0
19     20170907004465        3     5.0    -2.0
20     201709

In [ ]:
# defenders vs offenders
temp2 = train.merge(train.loc[train['IsRusher'],['PlayId','New_X','New_Y','Dir_std']]\
                  .rename(columns={'New_X':'Rusher_X','New_Y':'Rusher_Y','Dir_std':'Rusher_Dir'}), on = 'PlayId',how = 'left')
temp2[]

In [100]:
print(len(closest_op['PlayId'].unique()), len(best['PlayId'].unique()), len(temp['PlayId'].unique()))

23105 23105 23171


In [155]:
def split_data(df):
    df1 = df.loc[df['IsRusher']]
    df2 = df.loc[df['IsOnOffense'] & (~df['IsRusher'])]
    df3 = df.loc[~df['IsOnOffense']]

# max yards
    df1['Max_Yards'] = df1['YardLine']
    df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards']
    df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards']

# min_time_to_tacke
    df3['Min_Time_Tacke'] = df3['Distance_to_Rusher']/df3['S']
    df3.loc[df3['Min_Time_Tacke'] == np.inf, 'Min_Time_Tacke'] = 20

# defence_X_Y_spread
    Defence_X_Y_std = df3[["PlayId",'New_X','New_Y']].groupby("PlayId").std().rename(columns={'New_X':'Defense_X_std','New_Y':'Defense_Y_std'}) \
    .reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed2_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed2_std'}).reset_index()

    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed2_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed2_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed4_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed4_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed4_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Defence_X_Y_std, how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed4_std,how = 'left',  on ='PlayId')

#distance to QB
    dis_QB = df2.loc[df2["Position"] =='QB',['PlayId','Distance_to_Rusher']].groupby(['PlayId']).mean().rename(columns={'Distance_to_Rusher':'dis_to_QB'})
    df1 = df1.merge(dis_QB,how = 'left', on='PlayId')
#defence min,max,mean,std distance to rusher
    stat = df3.groupby(['GameId','PlayId']).agg({'Distance_to_Rusher':['min','max','mean','std']})
    stat.columns = stat.columns.droplevel()
    df1 = df1.merge(stat,how = 'left', on='PlayId')

# offense_X_Y_spread
    df2 = df2.sort_values(['PlayId','New_X'])
    Offense_X_Removed2_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed2_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed2_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed2_std'}).reset_index()
    
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_X_Removed4_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed4_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed4_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Offense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed4_std,how = 'left',  on ='PlayId')
    
# nearest offenders to defender
    dis_to_closest_offender = pd.DataFrame()
    dis_to_closest_defender = pd.DataFrame()
    for playid in df3['PlayId'].unique():
        offense = df2.loc[df2['PlayId'] == playid]
        defence = df3.loc[df3['PlayId'] == playid]
        ary = scipy.spatial.distance.cdist(defence[['New_X','New_Y']], offense[['New_X','New_Y']], metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, defence[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_offender = dis_to_closest_offender.append(ary)
        
        ary = scipy.spatial.distance.cdist(offense[['New_X','New_Y']], defence[['New_X','New_Y']],  metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, offense[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_defender = dis_to_closest_defender.append(ary)
    df3 = df3.merge(dis_to_closest_offender,how = 'left',on = ['PlayId','NflId'])
    df2 = df2.merge(dis_to_closest_defender,how = 'left',on = ['PlayId','NflId'])

#degree to closest offender
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df4 = df3.merge(df2, how = 'left', on = ['PlayId',0],suffixes=('_defender', '_offender'))
    df4['degree_to_closest_offender'] = df4[['New_X_offender','New_X_defender','New_Y_offender','New_Y_defender']].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df3['degree_to_closest_offender'] = df4['degree_to_closest_offender']
# personnel_features
    def defense_formation(l):
        dl = 0
        lb = 0
        db = 0
        other = 0

        for position in l:
            sub_string = position.split(' ')
            if sub_string[1] == 'DL':
                dl += int(sub_string[0])
            elif sub_string[1] in ['LB','OL']:
                lb += int(sub_string[0])
            else:
                db += int(sub_string[0])

        counts = (dl,lb,db,other)

        return counts
    def offense_formation(l):
        qb = 0
        rb = 0
        wr = 0
        te = 0
        ol = 0

        sub_total = 0
        qb_listed = False
        for position in l:
            sub_string = position.split(' ')
            pos = sub_string[1]
            cnt = int(sub_string[0])

            if pos == 'QB':
                qb += cnt
                sub_total += cnt
                qb_listed = True
            # Assuming LB is a line backer lined up as full back
            elif pos in ['RB','LB']:
                rb += cnt
                sub_total += cnt
            # Assuming DB is a defensive back and lined up as WR
            elif pos in ['WR','DB']:
                wr += cnt
                sub_total += cnt
            elif pos == 'TE':
                te += cnt
                sub_total += cnt
            # Assuming DL is a defensive lineman lined up as an additional line man
            else:
                ol += cnt
                sub_total += cnt

        # If not all 11 players were noted at given positions we need to make some assumptions
        # I will assume if a QB is not listed then there was 1 QB on the play
        # If a QB is listed then I'm going to assume the rest of the positions are at OL
        # This might be flawed but it looks like RB, TE and WR are always listed in the personnel
        if sub_total < 11:
            diff = 11 - sub_total
            if not qb_listed:
                qb += 1
                diff -= 1
            ol += diff

        counts = (qb,rb,wr,te,ol)

        return counts
    def split_personnel(s):
        splits = s.split(',')
        for i in range(len(splits)):
            splits[i] = splits[i].strip()

        return splits    
    def personnel_features(df):
        personnel = df[['GameId','PlayId','OffensePersonnel','DefensePersonnel']].drop_duplicates()
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: defense_formation(x))
        personnel['num_DL'] = personnel['DefensePersonnel'].apply(lambda x: x[0])
        personnel['num_LB'] = personnel['DefensePersonnel'].apply(lambda x: x[1])
        personnel['num_DB'] = personnel['DefensePersonnel'].apply(lambda x: x[2])

        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: offense_formation(x))
        personnel['num_QB'] = personnel['OffensePersonnel'].apply(lambda x: x[0])
        personnel['num_RB'] = personnel['OffensePersonnel'].apply(lambda x: x[1])
        personnel['num_WR'] = personnel['OffensePersonnel'].apply(lambda x: x[2])
        personnel['num_TE'] = personnel['OffensePersonnel'].apply(lambda x: x[3])
        personnel['num_OL'] = personnel['OffensePersonnel'].apply(lambda x: x[4])

        # Let's create some features to specify if the OL is covered
        personnel['OL_diff'] = personnel['num_OL'] - personnel['num_DL']
        personnel['OL_TE_diff'] = (personnel['num_OL'] + personnel['num_TE']) - personnel['num_DL']
        # Let's create a feature to specify if the defense is preventing the run
        # Let's just assume 7 or more DL and LB is run prevention
        personnel['run_def'] = (personnel['num_DL'] + personnel['num_LB'] > 6).astype(int)

        personnel.drop(['OffensePersonnel','DefensePersonnel'], axis=1, inplace=True)
        
        return personnel
    
    personnel = personnel_features(df1)   
    df1 = df1.merge(personnel,how = 'left',  on ='PlayId')
    
#select useful columns    
    rusher = df1[['PlayId','TimeDelta','Team','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y', \
                 'Orientation_std','Dir_std','Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position',\
                 'Quarter','GameClock','Down','Distance','OffenseFormation',\
                  'DefendersInTheBox','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                 'Offense_X_Removed2_std','Offense_Y_Removed2_std','Offense_X_Removed4_std','Offense_Y_Removed4_std',\
                 'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std','Defense_Y_Removed2_std','Defense_X_Removed4_std',\
                 'Defense_Y_Removed4_std',\
                  'num_DL','num_LB','num_DB','num_QB','num_RB','num_WR','num_TE','num_OL','OL_diff','OL_TE_diff','run_def',\
                 'min','max','std','mean','dis_to_QB','Max_Yards']]
    rusher = rusher.sort_values('PlayId')
    game = df1[['PlayId','Cleaned_GameWeather']]
    game = game.sort_values('PlayId')
    offender = df2[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff']]
    offender = offender.sort_values(['PlayId','Distance_to_Rusher'])
    defender = df3[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff','Min_Time_Tacke','Speed_Ratio','degree_to_closest_offender']+list(range(2))] #list(range(2)) two closest offenders
    defender = defender.sort_values(['PlayId','Distance_to_Rusher'])

    
    return rusher, game, offender, defender

In [156]:
rusher, game, offender, defender = split_data(train)

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [157]:
#remove some columns
rusher = rusher.drop('max',axis = 1)
rusher = rusher.fillna(-999)

offender = offender.sort_values(['PlayId','Distance_to_Rusher']).groupby('PlayId').head(2)

defender = defender.sort_values(['PlayId','Distance_to_Rusher']).drop(['New_Y','Position'],axis = 1).groupby('PlayId').head(6)

In [158]:
rusher = rusher.merge(closest_op, on='PlayId',how = 'left')
#rusher = rusher.merge(best, on='PlayId',how = 'left')
#rusher = rusher.merge(def_of, on='PlayId',how = 'left')

In [159]:
#encoding fit
le = preprocessing.LabelEncoder()
categories =[]
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = rusher[i].astype(str)
    categories.append(rusher[i].unique())

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = game[i].astype(str)
    categories.append(game[i].unique())

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = offender[i].astype(str)
    categories.append(offender[i].unique())

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = defender[i].astype(str)
    categories.append(defender[i].unique())
categories = np.hstack(categories)
le.fit(categories)
le.classes_

array(['-999', 'ACE', 'C', 'CB', 'DE', 'DT', 'EMPTY', 'FB', 'G', 'HB',
       'I_FORM', 'JUMBO', 'NT', 'OG', 'OLB', 'OT', 'PISTOL', 'QB', 'RB',
       'SHOTGUN', 'SINGLEBACK', 'T', 'TE', 'WILDCAT', 'WR', 'away',
       'home'], dtype=object)

In [160]:
#encoding transform
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = le.transform(rusher[i])

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = le.transform(game[i])

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = le.transform(offender[i])

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = le.transform(defender[i])

In [161]:
#reshape offender
offender_players = [offender.drop('PlayId',axis = 1).iloc[np.arange(k, len(offender), 2)].reset_index(drop = True) for k in range(2)]
offender_players = np.hstack([t.values for t in offender_players])
offender_players.shape

(23171, 32)

In [162]:
#reshape defender
defender_players = [defender.drop('PlayId',axis = 1).iloc[np.arange(k, len(defender), 6)].reset_index(drop = True) for k in range(6)]
defender_players = np.hstack([t.values for t in defender_players])
defender_players.shape


(23171, 114)

In [163]:
rusher.shape

(23171, 53)

In [164]:
#train
train_y =train.loc[train['IsRusher'],['Yards','PlayId']].sort_values('PlayId').drop('PlayId',axis = 1)
train_x = np.hstack([rusher.drop('PlayId',axis = 1).values,defender_players,offender_players])
train_y99 =(train_y + 99).reset_index(drop=True).Yards

In [165]:
#loss function
def crps_eval(y_pred, dataset, is_higher_better=False):
    labels = dataset.get_label()
    labels = labels.astype('int')
    y_true = np.zeros((len(labels),199))
    for i, v in enumerate(labels):
        y_true[i, v:] = 1
    y_pred = y_pred.reshape(-1, 199, order='F')
    y_pred = np.clip(y_pred.cumsum(axis=1), 0, 1)
    return 'crps', np.mean((y_pred - y_true)**2), False

In [166]:
# hyper parameters
params = {'max_depth':5, 'lambda_l1': 1.4, 'lambda_l2': 1.2,
 'num_leaves': 32, 'feature_fraction': 0.4,
 'subsample': 0.4, 'min_child_samples': 15,
 'learning_rate': 0.02,
 'num_iterations': 10000, 'random_state': 42,
 'objective': 'multiclass',
 'min_gain_to_split':0.9,
 'num_class':199,
 'metric':'None'}


In [167]:
import lightgbm as lgb
for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              valid_sets=[dtrain,dvalid],
                              early_stopping_rounds=10,
                              verbose_eval=100,
                              feval=crps_eval)

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds.
[100]	training's crps: 0.0126582	valid_1's crps: 0.0133339
[200]	training's crps: 0.0115313	valid_1's crps: 0.0130344
[300]	training's crps: 0.0108267	valid_1's crps: 0.0129014
[400]	training's crps: 0.0103728	valid_1's crps: 0.0128338
[500]	training's crps: 0.0100615	valid_1's crps: 0.0127975
[600]	training's crps: 0.00982425	valid_1's crps: 0.0127754
[700]	training's crps: 0.0096461	valid_1's crps: 0.0127629
[800]	training's crps: 0.00951129	valid_1's crps: 0.0127543
[900]	training's crps: 0.00942229	valid_1's crps: 0.0127486
Early stopping, best iteration is:
[902]	training's crps: 0.00942089	valid_1's crps: 0.0127483


C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds.
[100]	training's crps: 0.0127067	valid_1's crps: 0.0132604
[200]	training's crps: 0.0115757	valid_1's crps: 0.0129386
[300]	training's crps: 0.0108901	valid_1's crps: 0.0128041
[400]	training's crps: 0.0104595	valid_1's crps: 0.01274
[500]	training's crps: 0.010165	valid_1's crps: 0.0127007
[600]	training's crps: 0.0099332	valid_1's crps: 0.0126749
[700]	training's crps: 0.00975377	valid_1's crps: 0.0126572
[800]	training's crps: 0.00962831	valid_1's crps: 0.0126458
[900]	training's crps: 0.00954107	valid_1's crps: 0.0126406
Early stopping, best iteration is:
[954]	training's crps: 0.00950437	valid_1's crps: 0.0126372


C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds.
[100]	training's crps: 0.0127405	valid_1's crps: 0.0131439
[200]	training's crps: 0.0116386	valid_1's crps: 0.0128535
[300]	training's crps: 0.0109834	valid_1's crps: 0.012734
[400]	training's crps: 0.0105492	valid_1's crps: 0.0126779
[500]	training's crps: 0.0102446	valid_1's crps: 0.0126491
[600]	training's crps: 0.0100191	valid_1's crps: 0.0126302
[700]	training's crps: 0.00984445	valid_1's crps: 0.0126177
[800]	training's crps: 0.0097133	valid_1's crps: 0.0126096
[900]	training's crps: 0.00962434	valid_1's crps: 0.0126054
Early stopping, best iteration is:
[898]	training's crps: 0.00962544	valid_1's crps: 0.0126054


C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds.
[100]	training's crps: 0.0127249	valid_1's crps: 0.0131284
[200]	training's crps: 0.0116087	valid_1's crps: 0.012835


KeyboardInterrupt: 

In [115]:
#CV score
# for leave out one feature importance compare
print(np.mean([0.0129045,0.0128677,0.0127771,0.0127333,0.013174]))

# 5 closest defense palyer drop 'New_Y','back_oriented_down_field',max
print(np.mean([0.0128208, 0.0127364, 0.0126515, 0.0126222,0.0130614]))

# all defense palyer
print(np.mean([0.012802,0.0127486,0.0126934,0.0126226,0.0130668]))

# 5 closest defender + 2 closest offender l1,l2 = 1.2
print(np.mean([0.0127868,0.0127041,0.0126144,0.0125952,0.0130483]))

# 6 closest defender + 2 closest offender with new feature
print(np.mean([0.0127482, 0.0126271, 0.0126066, 0.0125627, 0.0130027]))


0.012891320000000001
0.01277846
0.01278668
0.012749759999999999
0.01270946


In [ ]:
from sklearn.model_selection import ParameterGrid

params_dic = {'max_depth':[5], 'lambda_l1': [1.8,2,2.2], 'lambda_l2': [1.8,2,2.2],
 'num_leaves': [32], 'feature_fraction': [0.35,0.4,0.45],
 'subsample': [0.40], 'min_child_samples': [15,20],
 'learning_rate': [0.015],
 'num_iterations': [10000], 'random_state': [42],
 'objective': ['multiclass'],
 'min_gain_to_split':[0.9,1],
 'num_class':[199],
 'metric':['None']}

for params in list(ParameterGrid(params_dic)):
    kfold = KFold(5, random_state = 42, shuffle = True)
    print(params)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              valid_sets=[dtrain,dvalid],
                              early_stopping_rounds=7,
                              verbose_eval=1000,
                              feval=crps_eval)

In [26]:
X_train = rusher.drop(['Defense_X_Removed4_std',\
                 'Defense_Y_Removed4_std'],axis = 1)
y_train = np.zeros(shape=(X_train.shape[0], 199))
for i,yard in enumerate(train_y.Yards):
    y_train[i, yard+99:] = np.ones(shape=(1, 100-yard))

In [27]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [28]:
np.isnan(offender_players).any()

NameError: name 'offender_players' is not defined

In [29]:
defender.isnull().sum()

PlayId                            0
PlayerAge                         0
PlayerHeight                      0
PlayerWeight                      0
New_X                             0
Orientation_std                   0
Dir_std                           0
Horizontal Speed                  0
Vertical Speed                    0
S                                 0
A                                 0
Dis                               0
Distance_to_Rusher                0
Degree_to_Rusher                  0
Degree_Diff                       0
Min_Time_Tacke                    0
Speed_Ratio                       0
degree_to_closest_offender    50966
0                                 0
1                                 0
dtype: int64

In [30]:
class RAdam(keras.optimizers.Optimizer):
    """RAdam optimizer.
    # Arguments
        learning_rate: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """

    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        learning_rate = kwargs.pop('lr', learning_rate)
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(learning_rate, name='learning_rate')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(min_lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = K.maximum(self.total_steps - warmup_steps, 1)
            decay_rate = (self.min_lr - lr) / decay_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr + decay_rate * K.minimum(t - warmup_steps, decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t))
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t))

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t >= 5, r_t * m_corr_t / (v_corr_t + self.epsilon), m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    @property
    def lr(self):
        return self.learning_rate

    @lr.setter
    def lr(self, learning_rate):
        self.learning_rate = learning_rate

    def get_config(self):
        config = {
            'learning_rate': float(K.get_value(self.learning_rate)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [31]:
#from https://www.kaggle.com/davidcairuz/nfl-neural-network-w-softmax
def crps(y_true, y_pred):
    return K.mean(K.square(y_true - K.cumsum(y_pred, axis=1)), axis=1)

In [33]:
def get_model():
    x = keras.layers.Input(shape=[X_train.shape[1]])
    fc1 = keras.layers.Dense(units=525, input_shape=[X_train.shape[1]])(x)
    act1 = keras.layers.ReLU()(fc1)
    bn1 = keras.layers.BatchNormalization()(act1)
    dp1 = keras.layers.Dropout(0.5)(bn1)
    gn1 = keras.layers.GaussianNoise(0.05)(dp1)
    concat1 = keras.layers.Concatenate()([x, gn1])
    fc2 = keras.layers.Dense(units=525)(concat1)
    act2 = keras.layers.ReLU()(fc2)
    bn2 = keras.layers.BatchNormalization()(act2)
    dp2 = keras.layers.Dropout(0.5)(bn2)
    gn2 = keras.layers.GaussianNoise(0.05)(dp2)
    concat2 = keras.layers.Concatenate()([concat1, gn2])
    fc3 = keras.layers.Dense(units=525)(concat2)
    act3 = keras.layers.ReLU()(fc3)
    bn3 = keras.layers.BatchNormalization()(act3)
    dp3 = keras.layers.Dropout(0.5)(bn3)
    gn3 = keras.layers.GaussianNoise(0.05)(dp3)
    concat3 = keras.layers.Concatenate([concat2, gn3])
    output = keras.layers.Dense(units=199, activation='softmax')(concat2)
    model = keras.models.Model(inputs=[x], outputs=[output])
    return model


def train_model(X_train, y_train, X_val, y_val):
    model = get_model()
    model.compile(optimizer=RAdam(warmup_proportion=0.1, min_lr=1e-7), loss=crps)
    er = EarlyStopping(patience=20, min_delta=1e-4, restore_best_weights=True, monitor='val_loss')
    model.fit(X_train, y_train, epochs=200, callbacks=[er], validation_data=[X_val, y_val], batch_size=batch_size)
    return model

In [ ]:
from sklearn.model_selection import RepeatedKFold
batch_size=32
rkf = RepeatedKFold(n_splits=5, n_repeats=5,random_state = 42)

models = []

for tr_idx, vl_idx in rkf.split(X_train, y_train):
    
    x_tr, y_tr = X_train[tr_idx], y_train[tr_idx]
    x_vl, y_vl = X_train[vl_idx], y_train[vl_idx]
    
    model = train_model(x_tr, y_tr, x_vl, y_vl)
    models.append(model)

Train on 18536 samples, validate on 4635 samples
Epoch 1/200
18536/18536 [==============================] - ETA: 5:27 - loss: 0.100 - ETA: 1:11 - loss: 0.097 - ETA: 38s - loss: 0.095 - ETA: 27s - loss: 0.09 - ETA: 21s - loss: 0.09 - ETA: 17s - loss: 0.09 - ETA: 15s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 10s - loss: 0.08 - ETA: 10s - loss: 0.08 - ETA: 9s - loss: 0.0880 - ETA: 9s - loss: 0.087 - ETA: 8s - loss: 0.086 - ETA: 8s - loss: 0.084 - ETA: 8s - loss: 0.083 - ETA: 7s - loss: 0.082 - ETA: 7s - loss: 0.080 - ETA: 7s - loss: 0.079 - ETA: 7s - loss: 0.077 - ETA: 6s - loss: 0.076 - ETA: 6s - loss: 0.075 - ETA: 6s - loss: 0.073 - ETA: 6s - loss: 0.072 - ETA: 6s - loss: 0.070 - ETA: 6s - loss: 0.069 - ETA: 6s - loss: 0.068 - ETA: 5s - loss: 0.067 - ETA: 5s - loss: 0.065 - ETA: 5s - loss: 0.064 - ETA: 5s - loss: 0.063 - ETA: 5s - loss: 0.062 - ETA: 5s - loss: 0.061 - ETA: 5s - loss: 0.060 - ETA: 5s - loss: 0.059 

Epoch 4/200
18536/18536 [==============================] - ETA: 5s - loss: 0.015 - ETA: 5s - loss: 0.015 - ETA: 5s - loss: 0.015 - ETA: 5s - loss: 0.015 - ETA: 5s - loss: 0.015 - ETA: 5s - loss: 0.016 - ETA: 5s - loss: 0.016 - ETA: 5s - loss: 0.016 - ETA: 5s - loss: 0.016 - ETA: 5s - loss: 0.016 - ETA: 5s - loss: 0.016 - ETA: 5s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 4s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 3s - loss: 0.016 - ETA: 

18536/18536 [==============================] - ETA: 6s - loss: 0.014 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.015 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 3s - loss: 0.014 - ETA: 3s - loss: 0.014 - ETA: 3s - loss: 0.014 - ETA: 3s - loss: 0.015 - ETA: 3s - loss: 0.015 - ETA: 3s - loss: 0

18536/18536 [==============================] - ETA: 6s - loss: 0.012 - ETA: 5s - loss: 0.016 - ETA: 5s - loss: 0.015 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.014 - ETA: 5s - loss: 0.013 - ETA: 5s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.014 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0

18536/18536 [==============================] - ETA: 6s - loss: 0.009 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.012 - ETA: 4s - loss: 0.013 - ETA: 4s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0.013 - ETA: 3s - loss: 0

...

18537/18537 [==============================] - ETA: 6s - loss: 0.013 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.012 - ETA: 6s - loss: 0.012 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0

18537/18537 [==============================] - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.014 - ETA: 6s - loss: 0.012 - ETA: 6s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.012 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0

18537/18537 [==============================] - ETA: 6s - loss: 0.016 - ETA: 6s - loss: 0.012 - ETA: 6s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0

18537/18537 [==============================] - ETA: 6s - loss: 0.008 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.010 - ETA: 4s - loss: 0.010 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.010 - ETA: 4s - loss: 0.010 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0

18537/18537 [==============================] - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0.011 - ETA: 4s - loss: 0